In [18]:
import boto3
from langchain_aws.llms import SagemakerEndpoint
from langchain_aws.llms.sagemaker_endpoint import LLMContentHandler
import json
 
sagemaker = boto3.client("sagemaker-runtime")

client = boto3.client("sagemaker-runtime")

class Llama38BContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"
 
    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 2046,
                "top_p": 0.9,
                "temperature": 0.6,
            },
        }
        input_str = json.dumps(
            payload,
        )
        # print(input_str)
        return input_str.encode("utf-8")
 
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        # print(response_json)
        content = response_json["generated_text"].strip()
        return content 

content_handler = Llama38BContentHandler()

llm = SagemakerEndpoint(
     endpoint_name="jumpstart-dft-llama-3-1-8b-instruct-20241121-063530",
     client=client,
    #  model_kwargs={"max_new_tokens": 30, "top_p": 0.9, "temperature": 0.7, "stop": ["\n"] },
     content_handler=content_handler)




In [19]:
example_query = "How many times does the letter 'e' appear in the word 'computer'?"
example_user = """{
#       "result": "3"
#     }"""
query = f"""You are a bot and you should reply to the user based on the function calling return. The return is {example_user}"""


In [20]:
from langchain.prompts import PromptTemplate

# Define the prompt template

prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{instruction}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["instruction", "question"]
)
example_instruct = "You are a bot and you should reply to the user based on the function calling return. The return is {'weather': 'Sunny', 'location': 'USA'}"
example_prompt = "what is the weather in USA now?"
prompt = PROMPT.format(instruction=example_instruct, question=example_prompt)
# Sample input to the prompt
# prompt_input = PROMPT.format(system=query, user=example_query)
# prompt_input = PROMPT.format(prompt_2)

# Invoke the model using the `llm` object
response = llm.invoke(prompt)

# Print the response
print("Response from model:")
print(response)

ERROR:root:Error raised by inference endpoint: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint jumpstart-dft-llama-3-1-8b-instruct-20241121-063530 of account 043309320622 not found.


ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint jumpstart-dft-llama-3-1-8b-instruct-20241121-063530 of account 043309320622 not found.